In [2]:
import numpy as np

# 勝敗と勝率の計算
def calculate_values(matrix):
    matrix = np.array(matrix)

    win = matrix[0, 0] + matrix[1, 1] + matrix[2, 2]
    lose = matrix[0, 2] + matrix[1, 0] + matrix[2, 1]
    draw = matrix[0, 1] + matrix[1, 2] + matrix[2, 0]

    total_sum = matrix.sum()
    accuracy = win * 100 / total_sum
    accuracy_2 = (win + lose) * 100 / total_sum
    
    return win, lose, draw, accuracy, accuracy_2

# 予測結果への対策
def my_hand(pred):
    if(isinstance(pred, np.ndarray)):
        pred = pred[0]
    
    if(pred == 0):
        opponent = "グー"
        me = "パー"
    elif(pred == 1):
        opponent = "チョキ"
        me = "グー"
    else:
        opponent = "パー"
        me = "チョキ"
    
    return opponent, me

# 今日の説明変数
today = [[0,1,4,3]]

In [3]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# データの読み込み
df = pd.read_csv('sazae_python.csv')

# 説明変数と目的変数の設定
data_orig = sklearn.utils.Bunch()
data_orig['target'] = df['sazae']
data_orig['data'] = df.loc[:, ['week_1','recent_G','recent_C','recent_P']]

## サポートベクターマシン（デフォルト）

In [4]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_orig['data'], data_orig['target'], test_size=0.2, shuffle=True, random_state=0)

# 説明変数を標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVMモデルの構築
model = SVC(kernel='linear', C=1)
model.fit(X_train_scaled, y_train)

# テストデータで予測
y_pred = model.predict(X_test_scaled)

# 精度の計算
win, lose, draw, accuracy, accuracy_2 = calculate_values(confusion_matrix(y_test, y_pred))
print("テストデータ：",win, "勝", lose, "敗", draw, "分（勝率：", accuracy, "％、負けない確率：", accuracy_2,"％）")

# 新規データ X_new を標準化
X_new_scaled = scaler.transform(today)

# SVMモデルを用いて予測
y_new_pred = model.predict(X_new_scaled)
opponent, me = my_hand(y_new_pred)
print("今日は", opponent, "を出してくるので", me, "を出してください")

テストデータ： 179 勝 71 敗 78 分（勝率： 54.573170731707314 ％、負けない確率： 76.21951219512195 ％）
今日は チョキ を出してくるので グー を出してください


/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## サポートベクターマシン（グリッドサーチ）

In [5]:
from sklearn.model_selection import GridSearchCV

# ハイパーパラメータの範囲を指定
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
}

# Grid Searchを使用して最適なハイパーパラメータを探索
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# 最適なハイパーパラメータを表示
print("Best Parameters:", grid_search.best_params_,"\n")

# 最適なモデルでテストデータで評価
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# 精度の計算
win, lose, draw, accuracy, accuracy_2 = calculate_values(confusion_matrix(y_test, y_pred))
print("テストデータ：",win, "勝", lose, "敗", draw, "分（勝率：", accuracy, "％、負けない確率：", accuracy_2,"％）")

# 新規データ X_new を標準化
X_new_scaled = scaler.transform(today)

# SVMモデルを用いて予測
y_new_pred = model.predict(X_new_scaled)
opponent, me = my_hand(y_new_pred)
print("今日は", opponent, "を出してくるので", me, "を出してください")

Best Parameters: {'C': 10, 'degree': 2, 'kernel': 'rbf'} 

テストデータ： 173 勝 81 敗 74 分（勝率： 52.74390243902439 ％、負けない確率： 77.4390243902439 ％）
今日は チョキ を出してくるので グー を出してください


/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## サポートベクターマシン（ランダムサーチ）

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# ハイパーパラメータの範囲を指定
param_dist = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
}

# Randomized Searchを使用して最適なハイパーパラメータを探索
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train_scaled, y_train)

# 最適なハイパーパラメータを表示
print("Best Parameters:", random_search.best_params_,"\n")

# 最適なモデルでテストデータで評価
best_model_random = random_search.best_estimator_
y_pred_random = best_model_random.predict(X_test_scaled)

# 精度の計算
win, lose, draw, accuracy, accuracy_2 = calculate_values(confusion_matrix(y_test, y_pred))
print("テストデータ：",win, "勝", lose, "敗", draw, "分（勝率：", accuracy, "％、負けない確率：", accuracy_2,"％）")

# 新規データ X_new を標準化
X_new_scaled = scaler.transform(today)

# SVMモデルを用いて予測
y_new_pred = model.predict(X_new_scaled)
opponent, me = my_hand(y_new_pred)
print("今日は", opponent, "を出してくるので", me, "を出してください")

Best Parameters: {'kernel': 'linear', 'degree': 2, 'C': 1} 

テストデータ： 173 勝 81 敗 74 分（勝率： 52.74390243902439 ％、負けない確率： 77.4390243902439 ％）
今日は チョキ を出してくるので グー を出してください


/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## ランダムフォレスト

In [7]:
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレストの構築
model = RandomForestClassifier(n_estimators=100, random_state=42)

# モデルの訓練
model.fit(X_train_scaled, y_train)

# テストデータで評価
y_pred = model.predict(X_test_scaled)

# 精度の計算
win, lose, draw, accuracy, accuracy_2 = calculate_values(confusion_matrix(y_test, y_pred))
print("テストデータ：",win, "勝", lose, "敗", draw, "分（勝率：", accuracy, "％、負けない確率：", accuracy_2,"％）")

# 新規データ X_new を標準化
X_new_scaled = scaler.transform(today)

# ランダムフォレストを用いて予測
y_new_pred = model.predict(X_new_scaled)
opponent, me = my_hand(y_new_pred)
print("今日は", opponent, "を出してくるので", me, "を出してください")

テストデータ： 176 勝 80 敗 72 分（勝率： 53.65853658536585 ％、負けない確率： 78.04878048780488 ％）
今日は チョキ を出してくるので グー を出してください


/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## ニューラルネットワーク

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# カテゴリ変数に変換
y_categorical = to_categorical(data_orig['target'])

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(data_orig['data'], y_categorical, test_size=0.2, random_state=42)

# 説明変数を標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ニューラルネットワークの構築
model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

# モデルのコンパイル
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# モデルの訓練
model.fit(X_train_scaled, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

# テストデータで評価
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# 精度の計算
win, lose, draw, accuracy, accuracy_2 = calculate_values(confusion_matrix(y_test_classes, y_pred_classes))
print("テストデータ：",win, "勝", lose, "敗", draw, "分（勝率：", accuracy, "％、負けない確率：", accuracy_2,"％）")

# 新規データ X_new を標準化
X_new_scaled = scaler.transform(today)

# ニューラルネットワークを用いて予測
y_new_pred = model.predict(X_new_scaled)
opponent, me = my_hand(np.argmax(y_new_pred))
print("今日は", opponent, "を出してくるので", me, "を出してください")

Epoch 1/25


/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5381 - loss: 1.0307 - val_accuracy: 0.5274 - val_loss: 0.9862
Epoch 2/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.5760 - loss: 0.9491 - val_accuracy: 0.5244 - val_loss: 1.0002
Epoch 3/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - accuracy: 0.5632 - loss: 0.9353 - val_accuracy: 0.5732 - val_loss: 0.9783
Epoch 4/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.5638 - loss: 0.9351 - val_accuracy: 0.5518 - val_loss: 0.9756
Epoch 5/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - accuracy: 0.5441 - loss: 0.9449 - val_accuracy: 0.5488 - val_loss: 0.9884
Epoch 6/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.5507 - loss: 0.9427 - val_accuracy: 0.5427 - val_loss: 0.9991
Epoch 7/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - accuracy: 0.5700 - loss: 0.9283 - val_accuracy: 0.5335 - val_loss: 1.0364
Epoch 8/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.5616 - loss: 0.9245 - val_accuracy: 0.5396 - val_lo

/Users/hashimototatsunori/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
